![iut](https://github.com/Hexanol777/STEM-Salaries-Case-Study/tree/main/Phase%201/stock_image/IUT200.png)
<hr style="margin-bottom: 40px;">


# STEM Jobs Salaries

## Data Aquisition, Cleaning and Wrangling

#### No data is clean right away! The data cleaning phase is a crucial step in any data analysis project as it involves identifying and correcting errors or inconsistencies in the data. This phase typically involves removing irrelevant data, handling missing values, standardizing data formats, and removing duplicates. Proper data cleaning ensures that our analysis is accurate and reliable and helps us draw meaningful insights and conclusions from the data.

[Link to the Data used in this Notebook](https://drive.google.com/file/d/1IhXv0qcq7YFfBxc0BQB1-z74wF40ZnZn/view?usp=share_link)

<img src="https://github.com/Hexanol777/STEM-Salaries-Case-Study/tree/main/Phase%201/stock_image/Process1.png"
    style="width:600px; float: bottom; margin: 1 80px 80px 80px;"></img>

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Data Extraction - Importing Modules

 During the data extraction phase, we obtained the data directly from [kaggle.com](www.kaggle.com), which is a popular platform for accessing and sharing datasets. By using Kaggle, we were able to search for and download datasets that were relevant to our analysis which in this case is STEM Jobs Salaries, and we could be confident in the quality of the data provided, as the usability of it was rated 10 in the website. Overall, the data extraction phase was streamlined and efficient, thanks to the availability and accessibility of high-quality data on Kaggle.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Loading The Initial Data:

In [ ]:
!head data/STEMJobs.csv
# Note: incase if you are running this line locally you will be met with the error below
# as this notebook is meant to be executed in Google Colab

In [ ]:
Data = pd.read_csv(
    'data/STEMJobs.csv',
    parse_dates=['timestamp'])

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Data First Look:

In [ ]:
Data.head()

In [ ]:
Data.info()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Data Cleaning

In the data cleaning phase, we removed certain race categories, such as `Race_Asian`, `Race_Black`, `Race_White`, `Race_Two_Or_More`, alongside the `educational attainment binaries` as they were not beneficial to our analysis. By streamlining the dataset, we get a step closer to an effective analysis:

In [ ]:
cols_to_drop = ['Race_Asian', 'Race_Black', 'Race_White', 'Race_Two_Or_More', 'Race_Hispanic','Race'
                , 'Masters_Degree', 'Bachelors_Degree', 'Doctorate_Degree', 'Highschool', 'Some_College'
               , 'tag', 'rowNumber', 'otherdetails', 'dmaid', 'cityid']

Data = Data.drop(columns=cols_to_drop)
Data.info()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Column Wrangling

The location in the "City, State" format. which is not relevant to our analysis, as we are interested in country-level insights. In order to improve our analysis, we need to homogenize the location data by creating a new column called 'Country'. This column will be derived from the existing location column and will only contain the 2-Alpha code of the country. By doing so, we can easily group and compare data at the country level, which will provide us with more meaningful insights for our analysis.

further on we will use the `pycountry` module which is big list of countries in the ISO 3166-1, ISO 4217 formats.
[Github Repository.](https://github.com/flyingcircusio/pycountry)


by using the `pycountry.subdivisions.lookup` method and a quick solution we can narrow down the process of changing every US city and state cell to just the 2-Alpha code 'US'. this easen the burden much since most of our data comes from the US. this method left us with only the name of the cities for other countries.

doing so we can rewrite the `get_country` function to now ignore the cells which have the 'US' value in them and use just the city names to figure out the 2-Alpha code of that country. for this matter the `geonamescache` [module](https://pypi.org/project/geonamescache/) will be used. this will only leave a small chunk of the data which can be manually edited to attain the finalized Data. 


In [ ]:
import pycountry

def get_country(location):
    city, state = location.split(", ")[:2] #leave only city and state name
    try:
        country = ''
        if pycountry.subdivisions.lookup(f'US-{state}'):
            country = "US"
    except LookupError:
            country = city
    return country

Frame = Data
# Apply the get_country function to the location column to create a new column for country names
Frame['Country'] = Frame['location'].apply(get_country)
Frame.head()

In [ ]:
import geonamescache

# Load the geonamescache data
gc = geonamescache.GeonamesCache()

def get_country_code(location):
    if location == "US":
        return "US"
    else:
        try:
            city = location
            country_dict = gc.get_cities_by_name(city)[0]
            second_part = list(country_dict.values())[0]
            country_code = second_part['countrycode']
        except (IndexError, KeyError):
            country_code = location
        return country_code
    
Geo_frame = Frame
Geo_frame['Country'] = Geo_frame['Country'].apply(lambda x: get_country_code(x) if x != "US" else "US").drop(columns='location')
Geo_frame.head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Finishing Touches


In [ ]:
# replace the 'Location' column with the column 'Country'
country = Geo_frame['Country']
Geo_frame['location'] = pd.DataFrame(country)

# last touches
Geo_frame.drop(columns=['Country'], inplace=True)
Geo_frame.rename(columns={'location': 'Country'})
Geo_frame.columns = [col.capitalize() for col in Geo_frame.columns]

# loop through each column and fill null values with "NA"
for col in Geo_frame.columns:
    Geo_frame[col].fillna(value='NA', inplace=True)
    
Geo_frame.head()
Geo_frame.shape

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Outlier Detection

Now that the data is clean and we have our desired columns we can run a outlier detection process for this matter the scipy module is used. Scipy is an open-source scientific computing library in Python that provides modules for scientific and technical computing.

In the code below, we first compute the z-scores for all the numerical columns of the DataFrame df. The z-score is a measure of how many standard deviations a data point is away from the mean of its column. We use np.abs() to take the absolute values of the z-scores, since we're interested in detecting both positive and negative outliers.

Next, we set a threshold above which a data point is considered an outlier. In this example, we chose a threshold of 3, which corresponds to a probability of less than 0.3% of a data point being that far away from the mean by chance, assuming a normal distribution.

In [ ]:
from scipy import stats

# specify columns to check for outliers
cols_to_check = ['Totalyearlycompensation', 'Basesalary', 'Stockgrantvalue', 'Bonus']

# define function to remove outliers from specified columns
def remove_outliers(Geo_frame, cols):
    for col in cols:
        z_scores = np.abs(stats.zscore(Geo_frame[col]))
        threshold = 3
        Geo_frame = Geo_frame[(z_scores < threshold) | (Geo_frame[col].isnull())]
    return Geo_frame

# remove outliers from specified columns
Geo_frame = remove_outliers(Geo_frame, cols_to_check)

Geo_frame.head()
Geo_frame.shape

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Outputting the Finalized Data?

Now that we have a finalized data frame, we need to save it to a .csv for future use. To do this, we will use the `to_csv()` method in pandas, which allows us to save the data frame to a CSV file format. To ensure that the file is successfully saved, we will use the `os` library in Python to check that the file exists and has the expected file size. This step is important to ensure that the data is saved correctly and can be used in future analyses.

In [ ]:
import os

PATH = 'data/jobs_with_country_codes_1.csv'
Geo_frame.to_csv(PATH, index=False)

# Check if the file exists
if os.path.exists(PATH):
    # Get the file details
    file_size = os.path.getsize(PATH)
    file_created = os.path.getctime(PATH)
    file_modified = os.path.getmtime(PATH)

    print(f"The file '{PATH}' exists.")
    print(f"Size: {file_size} bytes")
    print(f"Created: {file_created}")
    print(f"Last modified: {file_modified}")
    
else:
    print(f"The file '{file_path}' does not exist.")

## The Data is Now Ready to be Visualized!
![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
